<pre>
1. load the data

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. use cross entropy as loss function


</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>

</pre>

In [ ]:
file_path_ = "/content/"

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

from sklearn.model_selection import train_test_split

In [ ]:
tf.__version__  , tf.executing_eagerly()

('2.3.0', True)

In [ ]:
data_csv = pd.read_csv( file_path_ + "data.csv" , dtype={'label': 'int32' })

In [ ]:
data_csv.head(1)

,f1,f2,label
0,0.450564,1.074305,0


In [ ]:
data_csv.label.unique()

array([0, 1], dtype=int32)

In [ ]:
data_csv.groupby(['label']).agg({'label' : ['count'] })

,label
,count
label,
0,10000
1,10000


In [ ]:
# data_csv.drop( ['label'], axis=1)

In [ ]:
# data_csv
data_y = data_csv['label'].values
data_x = data_csv.drop( ['label'], axis=1).values
data_x.shape , data_y.shape 

((20000, 2), (20000,))

In [ ]:

#data_y = data_csv['label'].values
#data_x = data_csv.drop( ['label'], axis=1).values
# split the data into test and train by maintaining same distribution of output varaible 'data_y' [stratify=data_y]
X_train, X_test, Y_train, Y_test = train_test_split( data_x , data_y,stratify=data_y,test_size=0.20)


In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((16000, 2), (16000,), (4000, 2), (4000,))

In [ ]:
class LossHistory(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': []}
        
    def on_epoch_end(self, epoch, logs={}):
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['acc'].append(logs.get('acc'))
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_acc', -1) != -1:
            self.history['val_acc'].append(logs.get('val_acc'))

In [ ]:
class LearningRateCustom(tf.keras.callbacks.Callback):
      def __init__(self, schedule, verbose=0):
        super(LearningRateCustom, self).__init__()
        self.schedule = schedule
        self.verbose = verbose

      def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
          raise ValueError('Optimizer must have a "lr" attribute.')
        try:  # new API
          lr = float(K.get_value(self.model.optimizer.lr))
          lr = self.schedule(epoch, lr)
        except TypeError:  # Support for old API for backward compatibility
          lr = self.schedule(epoch)
        if not isinstance(lr, (ops.Tensor, float, np.float32, np.float64)):
          raise ValueError('The output of the "schedule" function '
                          'should be float.')
        if isinstance(lr, ops.Tensor) and not lr.dtype.is_floating:
          raise ValueError('The dtype of Tensor should be float')
        K.set_value(self.model.optimizer.lr, K.get_value(lr))
        if self.verbose > 0:
          print('\nEpoch %05d: LearningRateScheduler reducing learning '
                'rate to %s.' % (epoch + 1, lr))

      def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [ ]:
#Custom Callback class
class LearningRate(tf.keras.callbacks.LearningRateScheduler):
    def __init__(self ,  schedule, verbose=0 ):
        print("__init__ called")
        super(LearningRate, self ).__init__(  schedule, verbose )
        self.schedule = schedule
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        print("on LearningRate called")
       # self.history_data={'loss': [],'acc': [],'val_loss': [],'val_acc': []}
        self.initial_learningrate = 0.1
        
    def on_epoch_end(self, epoch, logs=None):
        ## on end of each epoch, we will get logs and update the self.history dict
        print("on_epoch_end called")
        print("logs : " , logs )
        self.schedule(epoch)
        '''self.history_data['loss'].append(logs.get('loss'))
        self.history_data['acc'].append(logs.get('accuracy'))
        if logs.get('val_loss', -1) != -1:
            self.history_data['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
            self.history_data['val_acc'].append(logs.get('val_accuracy'))

        # getting any NaN values(either weigths or loss) while training, it has to terminate your training.
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True
        '''


In [ ]:
#Custom Callback class
class LossHistoryData(tf.keras.callbacks.Callback):
    def __init__(self):
        print("__init__ LossHistoryData")

    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history_data
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        #print("on_train_begin called")
        self.history_data={'loss': [],'accuracy': [],'val_loss': [],'val_accuracy': []}
        
    def on_epoch_end(self, epoch, logs={}):
        ## on end of each epoch, we will get logs and update the self.history_data dict
        #print("on_epoch_end called")
        print("epoch:" , epoch)
        self.history_data['loss'].append(logs.get('loss'))
        self.history_data['accuracy'].append(logs.get('accuracy'))
        if logs.get('val_loss', -1) != -1:
            self.history_data['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
            self.history_data['val_accuracy'].append(logs.get('val_accuracy'))

        if (epoch > 2):
          val_acc = self.history_data.get("val_accuracy")
          if ( ( val_acc[epoch-1] < val_acc[epoch-2] ) and  ( val_acc[epoch-2] < val_acc[epoch-3]) ):
            #print("val_acc is not increased in last 2 epoch{}".format(val_acc[epoch-3 : epoch]))
            print("val_acc is not increased in last 2 epoch")
            self.model.stop_training = True
        # getting any NaN values(either weigths or loss) while training, it has to terminate your training.
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True

    #initial_learningrate=0.1
    def changeLearningRate4(self , epoch , lr ):
        #print( "---> changeLearningRate : " , epoch )
        #initial_learningrate=0.01
        val_loss = self.history_data.get("val_accuracy")
        #print(lr , "--->v_acc" , val_loss)
        #lr = ( lr ) *(0.1)
        if ( epoch > 1):
          if ( (epoch % 3) == 0):
            lr = ( lr ) *(0.05)
          elif (val_loss[epoch-1] < val_loss[epoch-2]):
            lr = ( lr ) *(0.1)
        
        return lr

In [ ]:
#initial_learningrate=0.1
def changeLearningRate(epoch , lr ):
        #print( "---> changeLearningRate : " , epoch )
        #initial_learningrate=0.01
        val_loss = history_own_data.history_data.get("val_accuracy")
        #print(lr , "--->v_acc" , val_loss)
        #lr = ( lr ) *(0.1)
        if ( epoch > 1):
          if ( (epoch % 3) == 0):
            lr = ( lr ) *(0.05)
          elif (val_loss[epoch-1] < val_loss[epoch-2]):
            lr = ( lr ) *(0.1)
        
        return lr

In [ ]:
#changeLearningRate(None, None)

In [ ]:
#Y_train = tf.keras.utils.to_categorical(Y_train, 10) 
#Y_test = tf.keras.utils.to_categorical(Y_test, 10)
Y_train = tf.keras.utils.to_categorical(Y_train, 2) 
Y_test = tf.keras.utils.to_categorical(Y_test, 2)


In [ ]:
#from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc , roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
class F1ScoreMetrics(tf.keras.callbacks.Callback):
    def __init__(self , validation ):   
        super(F1ScoreMetrics, self).__init__()
        self.validation_data = validation    
            
        print('validation shape', len(self.validation_data[0]))
        print("__init__F1ScoreMetrics")

    def on_train_begin(self, logs={}):        
        self.val_f1score = []
        self.auc_score = []
        #self.val_recalls = []
        #self.val_precisions = []
     
    def on_epoch_end(self, epoch, logs={}):
        #val_targ = self.validation[1]   
        #val_predict = (np.asarray(self.model.predict(self.validation[0]))).round()        
        val_x = self.validation_data[0]
        val_y = self.validation_data[1]
        val_predicted = (np.asarray(self.model.predict(val_x))).round() #(np.asarray()).round()
        #print("val_predicted : " , val_predicted.shape )
        #print("val_y : " , val_y.shape )
        val_f1 = f1_score(val_y, val_predicted ,  average='micro' )
        #val_recall = recall_score(val_targ, val_predict)         
        #val_precision = precision_score(val_targ, val_predict)
        #fpr,tpr,ths = roc_curve(np.mean(val_y, axis=1 ), np.mean( val_predicted , axis=1 ) )
        auc_sc = roc_auc_score( val_y, val_predicted , average='micro')
        #auc_sc = auc(fpr, tpr)
        
        self.val_f1score.append(round(val_f1, 6))
        self.auc_score.append( auc_sc )
        #self.val_recalls.append(round(val_recall, 6))
        #self.val_precisions.append(round(val_precision, 6))
 
        #print(f' — val_f1: {val_f1} — val_precision: {val_precision}, — val_recall: {val_recall}')
        print(f' — val_f1score: {val_f1} -- AUC : {auc_sc}' )

In [ ]:
import os

if (not os.path.isdir("/home/model_save")):
    print(os.makedirs("/home/model_save"))

In [ ]:
#to work with callback

#Input layer
input_layer = Input(shape=(2,))

#initializer = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
#Dense hidden layer
layer1 = Dense(50,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)

layer2 = Dense(40,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer1)
layer3 = Dense(30,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer2)
layer4 = Dense(20,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer3)
layer5 = Dense(10,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer4)

#output layer
output = Dense(10,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

optimizer = tf.keras.optimizers.Adam(0.01)
#opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
#Callbacks
history_own_data = LossHistoryData()
#history_own_data = LossHistory()
lrschedule = tf.keras.callbacks.LearningRateScheduler( changeLearningRate, verbose=1)
#lrschedule = LearningRate(changeLearningRate , verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data , lrschedule , f1score , checkpoint ]
#optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,Y_train,epochs=3, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 1/3
 987/1000 [============================>.] - ETA: 0s - loss: 0.7676 - accuracy: 0.4987epoch: 0
 — val_f1score: 0.5 -- AUC : 0.7222222222222221

Epoch 00001: val_loss improved from inf to 0.69575, saving model to /home/model_save/weights-01-0.5000.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.7666 - accuracy: 0.4991 - val_loss: 0.6958 - val_accuracy: 0.5000

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 2/3
 993/1000 [============================>.] - ETA: 0s - loss: 0.6953 - accuracy: 0.5023epoch: 1
 — val_f1score: 0.5 -- AUC : 0.7222222222222221

Epoch 00002: val_loss improved from 0.69575 to 0.69451, saving model to /home/model_save/weights-02-0.5000.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6954 - accuracy: 0.5019 - val_loss

In [ ]:
model.load_weights('/home/model_save/weights-02-0.5000.hdf5')
model.fit(X_train,Y_train)

500/500 [==============================] - 1s 2ms/step - loss: 0.6942 - accuracy: 0.4945


In [ ]:
history_own_data.history_data.get("val_accuracy")[0:4]

[0.5024999976158142,
 0.49300000071525574,
 0.47350001335144043,
 0.4737499952316284]

In [ ]:
history_own_data.history_data

{'acc': [0.5019375085830688, 0.5208125114440918, 0.6370000243186951],
 'loss': [0.7112691402435303, 0.6974461674690247, 0.6364936232566833],
 'val_acc': [0.5, 0.5137500166893005, 0.6667500138282776],
 'val_loss': [0.69703209400177, 0.6818374395370483, 0.6041727662086487]}

In [ ]:
history_own_data.history

{'acc': [None, None, None],
 'loss': [0.7149479985237122, 0.6955052018165588, 0.6351665258407593],
 'val_acc': [],
 'val_loss': [0.6971844434738159, 0.6879738569259644, 0.6043661236763]}

In [ ]:
#initial_learningrate=0.1
def changeLearningRate1(epoch , lr ):
        #print( "---> changeLearningRate : " , epoch )
        #initial_learningrate=0.01
        val_loss = history_own_data1.history_data.get("val_accuracy")
        #print(lr , "--->v_acc" , val_loss)
        #lr = ( lr ) *(0.1)
        if ( epoch > 1):
          if ( (epoch % 3) == 0):
            lr = ( lr ) *(0.05)
          elif (val_loss[epoch-1] < val_loss[epoch-2]):
            lr = ( lr ) *(0.1)
        
        return lr

In [ ]:
# model-1
#to work with callback 

#Input layer
input_layer1 = Input(shape=(2,))

initializer = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
#Dense hidden layer
layer1_1 = Dense(20,activation='tanh',kernel_initializer=initializer)(input_layer1)
layer1_2 = Dense(16,activation='tanh',kernel_initializer=initializer)(layer1_1)
layer1_3 = Dense(12,activation='tanh',kernel_initializer=initializer)(layer1_2)
layer1_4 = Dense(8,activation='tanh',kernel_initializer=initializer)(layer1_3)
layer1_5 = Dense(4,activation='tanh',kernel_initializer=initializer)(layer1_4)
#output layer
#output1 = Dense(10,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer1_5)
output1 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer1_5)

#Creating a model
model1 = Model(inputs=input_layer1,outputs=output1)

#optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
#optimizer_adam = tf.keras.optimizers.Adam(0.01)
#Callbacks
history_own_data1 = LossHistoryData()
lrschedule = tf.keras.callbacks.LearningRateScheduler( changeLearningRate1 , verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data1 , lrschedule , f1score , checkpoint]
#call_back_list = [history_own_data1 , f1score ]

model1.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model1.fit(X_train,Y_train,epochs=10, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 1: LearningRateScheduler setting learning rate to 0.10000000149011612.
Epoch 1/10
 978/1000 [============================>.] - ETA: 0s - loss: 0.7463 - accuracy: 0.4980epoch: 0
 — val_f1score: 0.0 -- AUC : 0.5

Epoch 1: val_loss improved from inf to 0.72060, saving model to /home/weights-01-0.5000.hdf5
1000/1000 [==============================] - 4s 3ms/step - loss: 0.7464 - accuracy: 0.4981 - val_loss: 0.7206 - val_accuracy: 0.5000 - lr: 0.1000

Epoch 2: LearningRateScheduler setting learning rate to 0.10000000149011612.
Epoch 2/10
 997/1000 [============================>.] - ETA: 0s - loss: 0.7301 - accuracy: 0.5028epoch: 1
 — val_f1score: 0.0 -- AUC : 0.5

Epoch 2: val_loss did not improve from 0.72060
1000/1000 [==============================] - 2s 2ms/step - loss: 0.7302 - accuracy: 0.5029 - val_loss: 0.7272 - val_accuracy: 0.5000 - lr: 0.1000

Epoch 3: LearningRateScheduler setting learning rate to 0.100

In [ ]:
#initial_learningrate=0.1
def changeLearningRate2(epoch , lr ):
        #print( "---> changeLearningRate : " , epoch )
        #initial_learningrate=0.01
        val_loss = history_own_data2.history_data.get("val_accuracy")
        #print(lr , "--->v_acc" , val_loss)
        #lr = ( lr ) *(0.1)
        if ( epoch > 1):
          if ( (epoch % 3) == 0):
            lr = ( lr ) *(0.05)
          elif (val_loss[epoch-1] < val_loss[epoch-2]):
            lr = ( lr ) *(0.1)
        
        return lr

In [ ]:
# model-2
#to work with callback

#Input layer
input_layer2 = Input(shape=(2,))

#initializer = tf.keras.initializers.he_uniform()
initializer =  tf.keras.initializers.RandomUniform(minval=0., maxval=1.)

#Dense hidden layer
layer2_1 = Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0., maxval=1.))(input_layer2)
layer2_2 = Dense(32,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0., maxval=1.))(layer2_1)
layer2_3 = Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0., maxval=1.))(layer2_2)
layer2_4 = Dense(8,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0., maxval=1.))(layer2_3)
layer2_5 = Dense(4,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0., maxval=1.))(layer2_4)
#output layer
#output2 = Dense(10,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer2_5)
output2 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer2_5)
#Creating a model
model2 = Model(inputs=input_layer2,outputs=output2)

#optimizer
optimizer2 = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
#Callbacks
history_own_data2 = LossHistoryData()
lrschedule = tf.keras.callbacks.LearningRateScheduler( changeLearningRate2 , verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data2 , lrschedule , f1score , checkpoint ]

model2.compile(optimizer=optimizer2 , loss='categorical_crossentropy',metrics=['accuracy'])

model2.fit(X_train,Y_train,epochs=3, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/3
 991/1000 [============================>.] - ETA: 0s - loss: 2.8816 - accuracy: 0.5050epoch: 0
 — val_f1score: 0.5 -- AUC : 0.5

Epoch 1: val_loss improved from inf to 0.69604, saving model to /home/weights-01-0.5000.hdf5
1000/1000 [==============================] - 3s 3ms/step - loss: 2.8619 - accuracy: 0.5057 - val_loss: 0.6960 - val_accuracy: 0.5000 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/3
 986/1000 [============================>.] - ETA: 0s - loss: 0.6940 - accuracy: 0.4963epoch: 1
 — val_f1score: 0.5 -- AUC : 0.5

Epoch 2: val_loss improved from 0.69604 to 0.69391, saving model to /home/weights-02-0.5000.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6940 - accuracy: 0.4966 - val_loss: 0.6939 - val_accuracy: 0.5000 - lr: 0.0100

Epoch 3: Le

In [ ]:
#initial_learningrate=0.1
def changeLearningRate3(epoch , lr ):
        #print( "---> changeLearningRate : " , epoch )
        #initial_learningrate=0.01
        val_loss = history_own_data3.history_data.get("val_accuracy")
        #print(lr , "--->v_acc" , val_loss)
        #lr = ( lr ) *(0.1)
        if ( epoch > 1):
          if ( (epoch % 3) == 0):
            lr = ( lr ) *(0.05)
          elif (val_loss[epoch-1] < val_loss[epoch-2]):
            lr = ( lr ) *(0.1)
        
        return lr

In [ ]:
# model-3
#to work with callback

#Input layer
input_layer3 = Input(shape=(2,))

initializer = tf.keras.initializers.he_uniform()
#Dense hidden layer
layer3_1 = Dense(30,activation='relu',kernel_initializer=initializer)(input_layer3)
layer3_2 = Dense(40,activation='relu',kernel_initializer=initializer)(layer3_1)
layer3_3 = Dense(30,activation='relu',kernel_initializer=initializer)(layer3_2)
layer3_4 = Dense(20,activation='relu',kernel_initializer=initializer)(layer3_3)
layer3_5 = Dense(10,activation='relu',kernel_initializer=initializer)(layer3_4)
#output layer
output3 = Dense(2,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer3_5)
#Creating a model
model3 = Model(inputs=input_layer3,outputs=output3)

#optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
#Callbacks
history_own_data3 = LossHistoryData()
lrschedule = tf.keras.callbacks.LearningRateScheduler( changeLearningRate3 , verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data3 , lrschedule , f1score , checkpoint ]

model3.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model3.fit(X_train,Y_train,epochs=3, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/3
 976/1000 [============================>.] - ETA: 0s - loss: 0.6379 - accuracy: 0.6391epoch: 0
 — val_f1score: 0.3134872417982989 -- AUC : 0.57625

Epoch 1: val_loss improved from inf to 0.62453, saving model to /home/model_save/weights-01-0.6585.hdf5
1000/1000 [==============================] - 5s 4ms/step - loss: 0.6378 - accuracy: 0.6395 - val_loss: 0.6245 - val_accuracy: 0.6585 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/3
 991/1000 [============================>.] - ETA: 0s - loss: 0.6168 - accuracy: 0.6609epoch: 1
 — val_f1score: 0.30837181288897814 -- AUC : 0.569375

Epoch 2: val_loss improved from 0.62453 to 0.59708, saving model to /home/model_save/weights-02-0.6737.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6175 - accuracy: 0.6606 - val

In [ ]:
# model-4
#to work with callback

#Input layer
input_layer4 = Input(shape=(2,))

initializer = tf.keras.initializers.he_uniform()
#Dense hidden layer
layer4_1 = Dense(100,activation='relu',kernel_initializer=initializer)(input_layer4)
layer4_2 = Dense(64,activation='relu',kernel_initializer=initializer)(layer4_1)
layer4_3 = Dense(32,activation='relu',kernel_initializer=initializer)(layer4_2)
layer4_4 = Dense(8,activation='relu',kernel_initializer=initializer)(layer4_3)
layer4_5 = Dense(4,activation='relu',kernel_initializer=initializer)(layer4_4)
#output layer
output4 = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer4_5)
#Creating a model
model4 = Model(inputs=input_layer4,outputs=output4)

#optimizer
#optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
##optimizer_adam = tf.keras.optimizers.Adam(0.01)
#Callbacks
history_own_data4 = LossHistoryData()
lrschedule = tf.keras.callbacks.LearningRateScheduler( history_own_data4.changeLearningRate4 , verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data4 , lrschedule , f1score , checkpoint ]

model4.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

model4.fit(X_train,Y_train,epochs=10, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10
 988/1000 [============================>.] - ETA: 0s - loss: 0.6651 - accuracy: 0.5998epoch: 0
 — val_f1score: 0.66125 -- AUC : 0.6612499999999999

Epoch 1: val_loss improved from inf to 0.62278, saving model to /home/model_save/weights-01-0.6612.hdf5
1000/1000 [==============================] - 4s 3ms/step - loss: 0.6647 - accuracy: 0.5999 - val_loss: 0.6228 - val_accuracy: 0.6612 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/10
 994/1000 [============================>.] - ETA: 0s - loss: 0.6169 - accuracy: 0.6634epoch: 1
 — val_f1score: 0.66425 -- AUC : 0.66425

Epoch 2: val_loss improved from 0.62278 to 0.60193, saving model to /home/model_save/weights-02-0.6643.hdf5
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6168 - accuracy: 0.6633 - val_loss: 0.